In [1]:
#import requests

In [2]:
#standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
#data = requests.get(standings_url)

In [4]:
#from bs4 import BeautifulSoup

In [5]:
#soup = BeautifulSoup(data.text)

In [6]:
#standings_table = soup.select('table.stats_table')[0]

In [7]:
#links = standings_table.find_all('a')

In [8]:
#links = [l.get("href") for l in links]

In [9]:
#links = [l for l in links if '/squads/' in l]

In [10]:
#team_urls = [f"https://fbref.com{l}" for l in links]

In [11]:
#team_url = team_urls[0]

In [12]:
#data = requests.get(team_url)

In [13]:
"""import pandas as pd
from io import StringIO
html_content = StringIO(data.text)
matches = pd.read_html(html_content, match="Scores & Fixtures")
"""

'import pandas as pd\nfrom io import StringIO\nhtml_content = StringIO(data.text)\nmatches = pd.read_html(html_content, match="Scores & Fixtures")\n'

In [14]:
#matches[0].head()

In [15]:
#soup = BeautifulSoup(data.text)

In [16]:
#links = soup.find_all('a')

In [17]:
#links = [l.get("href") for l in links]

In [18]:
#links = [l for l in links if l and 'all_comps/shooting/' in l]

In [19]:
#links

In [20]:
#data = requests.get(f"https://fbref.com{links[0]}")

In [21]:
#html_content = StringIO(data.text)
#shooting = pd.read_html(html_content, match="Shooting")[0]

In [22]:
#shooting.head(5)

In [23]:
#shooting.columns = shooting.columns.droplevel()

In [24]:
#shooting.head()

In [25]:
#team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [26]:
#team_data.head()

In [27]:
years = list(range(2024, 2016, -1))

In [28]:
all_matches = []

In [29]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [30]:
import time
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup
import requests

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        data = requests.get(team_url)
        html_content = StringIO(data.text)
        matches = pd.read_html(html_content, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        html_content = StringIO(data.text)
        shooting=pd.read_html(html_content, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(7)
        

In [31]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv('17-24-matches.csv')